### **Objective:** 

The objective of this project is to segment the customers of a retail company based on their purchasing behaviour.

### **Data analysis process:**

Analysis will be done as follows.

1. Understanding data
2. Data cleaning (Removing duplicates, treating outiliers and missing values etc.)
3. RFM analysis to segment the customers based on their purchasing behaviour.
4. K-Means clustering.
5. Interpretation of the clusters.

In [2]:
# Load the following libraries.

import numpy as np
import datetime as dt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
!pip install openpyxl
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

%matplotlib inline

In [3]:
# Load the data.

customer_df = pd.read_excel("online_retail_II.xlsx")

customer_df

In [4]:
# Check the data.

customer_df.dtypes

In [5]:
# Check the data.

customer_df.describe() 

# There are negative values which indicate returned products.

In [6]:
# Check the data.

customer_df['Country'].value_counts()

# Majority of transcations are from United Kingdom.

In [7]:
# Check the data for missing values.

customer_df.isnull().sum()

# There are many customers without Customer ID.

In [8]:
# Renaming "Customer ID" as "CustomerID".

customer_df = customer_df.rename(columns = {"Customer ID": "CustomerID"})

In [9]:
# Checking correlation between variables.

ax = sns.heatmap(customer_df.corr(), annot=True)

# Variables are not correlated with each other.

In [10]:
# Removing negative values.

customer_df = customer_df[(customer_df['Quantity'] > 0) & (customer_df['Price'] > 0)] 

customer_df.describe()

In [11]:
# Removing duplicates.

customer_df = customer_df.drop_duplicates()

customer_df.shape

### **RFM Analysis**

In [12]:
# Calculating total cost and storing it as "Amount".

customer_df['Amount'] = customer_df['Quantity'] * customer_df['Price']

In [13]:
# Finding out last Invoice date to calculate "Recency".

customer_df.InvoiceDate.max()

In [14]:
# Creating RFM dataframe.

pin_date = dt.datetime(2010, 12,10)

rfm = customer_df.groupby('CustomerID').agg({'InvoiceDate': lambda InvoiceDate: (pin_date - InvoiceDate.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'Amount': lambda Amount: Amount.sum()})

rfm.head()

In [15]:
# Changing column names.

rfm.columns = ['recency', 'frequency', 'monetary']

rfm.head()

### **Calculating the score**

We need to assign a score from 1 to 5 to recency, frequency and monetary value individually for each customer.

Below is the conversion of columns into rfm scores between 1 to 5.

'5' being the highest and '1' being the least.

* The higher the monetary value, higher is the score i.e.'5'.

* Smaller value of recency indicates recent purchases. So it takes the higher value of 5.

* Frequency is the same as monetary, higher the frequency, higher the score.

In [16]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels = [5, 4, 3, 2, 1])

rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method = "first"), 5, labels = [1, 2, 3, 4, 5])

rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels = [1, 2, 3, 4, 5])

rfm['score'] = rfm['recency_score'].astype(int)+rfm['frequency_score'].astype(int)+rfm['monetary_score'].astype(int)

rfm.head()

### **Identifying best customers using score.**

In [17]:
# Analysing "rfm" dataframe.

rfm.describe()

# Customers with score of 15 are best customers as they have have the highest recency, frequency and monetary score.

### **Seggregate the levels** 

We are going to seggregate the level of the customer based on the score.

Here we have classified them into 4 levels.

* Active: High revenue generating and frequent buyers.

* Good: Customers whose purchases are fairly frequent and generate moderate revenue.

* Average: Customers who are less active and are not very frequent buyers and generate low revenue.

* Inactive: Customers generating very low revenue and are occasional buyers.

In [18]:
# Definig a function to assign levels to customers.

def rfm_level(score):
    if  ((score >1) and (score < 4)):
        return 'Inactive'
    elif ((score >3) and (score < 8)):
        return 'Average'
    elif ((score > 7) and (score <11)):
        return 'Good'
    else:
        return 'Active'

In [19]:
# Assigning levels to customers.

rfm['level'] = rfm['score'].apply(lambda score : rfm_level(score))

rfm.head()

In [20]:
# Visualizing the number of customers in every level.

plt.figure(figsize = (10,5))

sns.set_context("poster", font_scale = 0.7)

sns.countplot(rfm['level'])

# Number of active customers are more in the dataset.

In [21]:
# Analysing customers in each level.

rfm.groupby('level').agg({
    'recency': ['mean', 'min','max'],
    'frequency': ['mean', 'min','max'],
    'monetary': ['mean','min','max','count']
})

### **Active Customers:**

We can say that 1597 customers bought 4540.46 units by shopping 9 times approximately every 26 days.

They are previliged customers.

Surprise offers can be given on birthdays and anniversary.

### **Good Customers:**

1079 customers have bought 939.92 units by shopping thrice every 70 days

They are loyal customers.

### **Average customers:**

1434 customers have bought approximately 351.56 units by shopping once every 151 days.

Tiered pricing of products can be introduced.

### **Inactive customers:**

202 customers have bought approximately 142.49 units by shopping once every 273 days.

Personalised emails and SMS can be sent to the customers regarding the offers.

More marketing strategies can be introduced to improve the frequency of orders.

In [22]:
# Segmenting customers based on their levels.

active = rfm[rfm['level'] == 'Active']

average = rfm[rfm['level'] == 'Average']

good = rfm[rfm['level'] == 'Good']

inactive = rfm[rfm['level'] == 'Inactive']

In [23]:
# Saving segements in different excel files.

# We are segmenting customers based on their levels, so that it will be easy for the company to send relevent offers and messages to group of customers who belong to the same segment.

active_df = pd.DataFrame()

active_df["customer_id"] = rfm[rfm["level"] == "Active"].index

active_df.to_excel("active_customers.xlsx", sheet_name='Active Customers Index')


average_df = pd.DataFrame()

average_df["customer_id"] = rfm[rfm["level"] == "Average"].index

average_df.to_excel("average_customers.xlsx", sheet_name='Average Customers Index')


good_df = pd.DataFrame()

good_df["customer_id"] = rfm[rfm["level"] == "Good"].index

good_df.to_excel("good_customers.xlsx", sheet_name='Good Customers Index')


inactive_df = pd.DataFrame()

inactive_df["customer_id"] = rfm[rfm["level"] == "Inactive"].index

inactive_df.to_excel("inactive_customers.xlsx", sheet_name='Inactive Customers Index')

### **K-Means clustering**

Applying K-Means method to identify different clusters based on segmentation of customers according to their purchasing behaviour.

Clustering will be done using 3 factors as recency, frequency and monetary values.

### **Outlier treatment**

In [24]:
# Extracting only numeric columns from rfm.

numeric_cols = rfm.select_dtypes(include = 'number')

In [25]:
# Plotting boxplots for all continous variables in order to detect outliers.

for column in numeric_cols:
    plt.figure(figsize = (10,5))
    numeric_cols.boxplot([column])
    
# Some variables have outliers. These outliers will be treated in the next step.

In [26]:
# Flooring and capping ouliers.

for col in numeric_cols.columns:
    Q1 = numeric_cols[col].quantile(0.25)
    Q3 = numeric_cols[col].quantile(0.75)
    IQR = Q3 - Q1 
    Lower_cap  = Q1 - 1.5*IQR
    Upper_cap = Q3 + 1.5*IQR
    numeric_cols[col][numeric_cols[col] <= Lower_cap] = Lower_cap
    numeric_cols[col][numeric_cols[col] >= Upper_cap] = Upper_cap

In [27]:
# Dropping following columns from rfm.

rfm.drop(['recency', 'frequency', 'monetary', 'score'], axis = 1, inplace = True)

In [28]:
# Setting indices.

numeric_cols.reset_index(drop = True, inplace = True)

rfm.reset_index(drop = True, inplace = True)

In [29]:
# Joining df "numeric_cols" and df "rfm".

rfm = pd.concat([rfm, numeric_cols], axis = 1)

rfm

In [30]:
# Scaling the data before applying K-Means.

rfm1 = rfm[['recency','frequency','monetary']]

scaler = StandardScaler()

scaled = scaler.fit(rfm1)

scaled = scaler.fit_transform(rfm1)

scaled

In [31]:
# Finding out number of clusters to be formed using Elbow method.

model = KMeans()

visualizer = KElbowVisualizer(model, k = (1,12))

visualizer.fit(scaled)

visualizer.show()   

# According to Elbow method, number of clusters = 3.

In [32]:
# Applying K-Means.

kmeans_scaled = KMeans(3)

kmeans_scaled.fit(scaled)

identified_clusters = kmeans_scaled.fit_predict(rfm1)

clusters_scaled = rfm1.copy()

clusters_scaled['cluster_pred'] = kmeans_scaled.fit_predict(scaled)

print(identified_clusters)

sns.set(style = "darkgrid")

print(" Our cluster centers are as follows")

print(kmeans_scaled.cluster_centers_)

f, ax = plt.subplots(figsize = (25, 5))

ax = sns.countplot(x = "cluster_pred", data = clusters_scaled)

clusters_scaled.groupby(['cluster_pred']).count()

In [33]:
# Visualizing the clusters.

fig = plt.figure()

ax = plt.axes(projection = '3d')

xline = clusters_scaled['recency']

yline = clusters_scaled['frequency']

zline = clusters_scaled['monetary']

ax.scatter3D(xline, zline,yline,c = clusters_scaled['cluster_pred'])

ax.view_init(30, 60)

### **Cluster profiling**

In [34]:
# Analyzing the clusters.

rfm1['cluster'] = clusters_scaled['cluster_pred']

rfm1['level'] = rfm['level']

rfm1.groupby('cluster').agg({
    'recency' : ['mean','min','max'],
    'frequency' : ['mean','min','max'],
    'monetary' : ['mean','min','max','count']
})

Cluster 1: Not good customers as they have highest recency and lowest frequency and monetary values.

Cluster 2: It has best customers as it has least recency and highest frequency and monetary values.



In [35]:
# Checking the nature of the clusters.

rfm1.groupby(['cluster','level']).size()

# Cluste 0 and 1 are heterogeneous whereas cluster 2 is homogeneous in nature.

### **Conclusions:**

Two kinds of segmentations as RFM analysis and K-Means clustering were performed.

Customers have been categorized into 4 groups based on Recency, Frequency and Monetary value of their purchases as active, good, average and inactive customers. 

By applying K-Means clustering,

* We were able to find out best customers,i.e. the most valuable and profitable group of customers.
* This enables us to determine which customer group should be targeted and to whom to give special offers or promotions so as to increase the revenue of the company.
* Based on above information, we can come up with improved marketing strategies and best communication channel for each segment of customers.